In [1]:
import itertools
import numpy as np
import pandas as pd
import time

In [2]:
def n_div(n,h):
    col = []
    while len(col) != h:
        col.append(int(n/(h)))
    i = -1
    while sum(col) != n:
        col[i] = col[i] + 1
        i -= 1
    return(col)

In [3]:
def matrix(h):
    if h == 2:
        return([[1,2],[2,1]])
    total_node = (2**(h+1))-2
    col_sums = n_div(total_node,h)
    lt = [0]*h
    lt[0] = 1
    lt[1] = col_sums[1]-1
    reflex_node = int((total_node - (sum(lt)*2))/4)
    for i in range(2,h):
        lt[i] = int(col_sums[i]/2)
    rt = lt[2:].copy()
    rt.reverse()
    rt.insert(0,1)
    rt.insert(0,lt[1])
    lt_id = 2
    rt_id = -1
    if sum(lt) - sum(rt) == 1:
        rt[-1] +=1
    if sum(rt) - sum(lt) == 1:
        lt[2] +=1    
    while sum(lt)+sum(rt)!= total_node:
        lt[lt_id] += 1
        rt[rt_id] += 1
        lt_id += 1
        rt_id -= 1
    lt = np.array(lt,dtype=float)
    rt = np.array(rt,dtype=float)
    
    return(lt,rt)

In [4]:
#n colours
#each array has size n for each color
def split(number, array, set_list):
    #print(array)
    #numpy array
    n = array.size
    unique, counts = np.unique(array, return_counts=True)
    X = dict(zip(unique, counts))
    assert X[1] == 1
    if array[np.where(~np.isinf(array))].sum() <= 1 + 2: #base case or 1 + 2 + 4
        parent = np.where(array==1)[0][0]
        leaf = np.where(array==2)[0][0]
        set_list[parent].append(number)
        set_list[leaf].append(number*2)
        set_list[leaf].append(number*2+1)
        return

    new_sum = array[np.where(~np.isinf(array))].sum() // 2
    
    #remove 1
    index_1 = np.where(array==1)[0][0]
    array[index_1] = np.inf
    
    non_zero = np.nonzero(array)
    
    result1 = np.ones(n)*np.inf
    result2 = np.ones(n)*np.inf
    small_val = array.min()
    min_index = np.where(array==small_val)[0][0]
    result1[min_index] = 1
    result2[min_index] = array[min_index] - result1[min_index]
    if small_val > 2:
        #1 has to be in a different position
        sec_val = np.sort(array)[1]
        if sec_val == small_val:
            sec_index = np.where(array==sec_val)[0][1]
        else:
            sec_index = np.where(array==sec_val)[0][0]

        assert sec_index != min_index
        result2[sec_index] = 1
        result1[sec_index] = array[sec_index] - result2[sec_index]
    
    f=False
    for i in range(n):
        if i != min_index and (small_val == 2 or i != sec_index) and not np.isinf(array[i]):
            f = True
            result1[i] = array[i]//2
            result2[i] = array[i] - result1[i]
            changed_index = i
    if result1[np.where(~np.isinf(result1))].sum() != new_sum:
        if f==False and small_val>2:
            changed_index = sec_index
        #print(result1)
        #print(result1, new_sum, changed_index, f, min_index, sec_index)
        result1[changed_index] += new_sum - result1[np.where(~np.isinf(result1))].sum().sum()
        result2[changed_index] = array[changed_index] - result1[changed_index]
    set_list[index_1].append(number)
    return split(number*2, result1,set_list), split(number*2+1, result2,set_list)

In [5]:
def part_the_sea(h):
    if h != 2:
        colours = []
        for i in range(h):
            colours.append([])
        half_tree_l,half_tree_r = matrix(h)
        split(2,half_tree_l,colours)
        split(3,half_tree_r,colours)
        for i in range(len(colours)):
            colours[i] = sorted(colours[i])
    else:
        colours = [[2,6,7],[3,4,5]]
    file_name = 'test'+str(h)+'.txt'
    with open(file_name, 'w') as f:
        f.write(str(h)+'\n')
        for line in colours:
            f.write(' '.join([str(x) for x in line])+'\n')

## Use this to generate a txt file of unrelated partions of size h

In [ ]:
part_the_sea(2)